# analysis V4 - new basis code - Dec 6, 2023
- cleaned up the data QC and organizations a bit
- having two anndata and their spin offs - both FISH and proj data 
- flip an axis (y-axis) when plotting - not in data
- focus on L2/3 cells

TODO: 
- separate and organize plotting functions 
- organize the plots and generate more insights

In [ ]:
import numpy as np
import pandas as pd
import os 
import matplotlib.pyplot as plt
import umap
import seaborn as sns
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
import matplotlib

import anndata 

from scroutines import config_plots
from scroutines import basicu
from scroutines import miscu

In [ ]:
import sys
sys.setrecursionlimit(10000)
from scipy.cluster import hierarchy as sch

In [ ]:
def norm_data(adata):
    """This procedure is independnet of each gene
    norm by size (cell volume)
    norm by log2(1+)
    norm by zscore
    """
    # size
    med_size = adata.obs['area'].median()
    max_size = adata.obs['area'].max()
    min_size = adata.obs['area'].min()

    print(f"Med cell size {med_size:.1f} um^3\t  {np.power(med_size,1/3):.1f} um")
    print(f"Max cell size {max_size:.1f} um^3\t  {np.power(max_size,1/3):.1f} um")
    print(f"Min cell size {min_size:.1f} um^3\t  {np.power(min_size,1/3):.1f} um")

    size_factor = (adata.obs['area']/med_size).values


    # norm by size; by log2+1; by zscore
    mat_raw = np.array(adata.X)
    mat_nrm = mat_raw/size_factor.reshape(-1,1) # .divide(size_factor, axis=0)
    mat_log = np.log2(1+mat_nrm)
    mat_zsc = (mat_log - np.mean(mat_log, axis=0))/(1e-10+np.std(mat_log, axis=0))

    adata.layers['nrm'] = mat_nrm
    adata.layers['log'] = mat_log
    adata.layers['zsc'] = mat_zsc
    
    return # mat_raw, mat_nrm, mat_log, mat_zsc

In [ ]:
pth_dat = '/u/home/f/f7xiesnm/project-zipursky/easifish/lt186/proc/r23467v3/'
!ls $pth_dat

In [ ]:
# features
var_names = {
    # 'r1_c0': "RL (Cre)",
    # 'r1_c2': "LM (FlpO)",
    'r2v3_c0': 'Sorcs3',
    'r2v3_c1': 'Kcnq5',
    'r2v3_c2': 'Chrm2',
    'r2v3_c4': 'Adamts2',
    
    'r3v3_c0': 'Kcnip3',
    'r3v3_c1': 'Rorb',
    'r3v3_c2': 'Cdh13',
    'r3v3_c4': 'Cntn5',
    
    'r4v3_c0': 'Cdh12',
    'r4v3_c1': 'Gria3',
    'r4v3_c2': 'Cntnap2',
    'r4v3_c4': 'Gabrg3',
    
    'r6v3_c0': 'Ncam2',
    'r6v3_c1': 'Rfx3',
    'r6v3_c2': 'Epha10',
    'r6v3_c4': 'Baz1a',
    
    'r7v3_c0': 'Kcnh5',
    'r7v3_c1': 'Slc17a7',
    'r7v3_c2': 'Trpc6',
    'r7v3_c4': 'Grm8',
}
proj_idx = np.array([]) #np.array(['r1_c0', 'r1_c2'])

# cells
f_meta = os.path.join(pth_dat, 'roi.csv')

# spots
f_spot = os.path.join(pth_dat, 'spotcount.csv')

In [ ]:
# f = '/u/home/f/f7xiesnm/project-zipursky/v1-bb/v1/data/cheng21_cell_scrna/res/L23-ABC-genes-n288-n286unq-annot_v2.csv'
f = '/u/home/f/f7xiesnm/project-zipursky/v1-bb/v1/results/gene_ptime_P28_L23_Mar27.tsv'
df_annot = pd.read_csv(f).sort_values('gene_ptime')

In [ ]:

var = pd.Series(var_names).to_frame('name')
var['proj'] = False
var.loc[proj_idx, 'proj'] = True
var['ptime'] = df_annot.set_index('gene').reindex(var['name'])['gene_ptime'].values
var['ptime_order'] = var['ptime'].rank()  

var_idx = var.index.values.astype(str)
var_i2n = var['name'] 
var_n2i = var.reset_index().set_index('name')['index']

gene_idx = np.array([idx for idx in var_idx if idx not in proj_idx])

# ftrs_order  = gene_names.index.values 
# genes_order = gene_names.values
var

In [ ]:
# cols
raw_var_idx = np.char.add(var_idx, '_raw')
nrm_var_idx = np.char.add(var_idx, '_nrm')
log_var_idx = np.char.add(var_idx, '_log')
zsc_var_idx = np.char.add(var_idx, '_zsc')

raw_gene_idx = np.char.add(gene_idx, '_raw')
nrm_gene_idx = np.char.add(gene_idx, '_nrm')
log_gene_idx = np.char.add(gene_idx, '_log')
zsc_gene_idx = np.char.add(gene_idx, '_zsc')



In [ ]:
meta = pd.read_csv(f_meta, index_col=0)
spot = pd.read_csv(f_spot, index_col=0)
assert np.all(meta.index.values == spot.index.values)
print(meta.shape, spot.shape) # 

max_x, max_y, max_z = meta[['x', 'y', 'z']].describe().loc['max']
min_x, min_y, min_z = meta[['x', 'y', 'z']].describe().loc['min']
print(f'x: {min_x:.1f}\t{max_x:.1f}')
print(f'y: {min_y:.1f}\t{max_y:.1f}')
print(f'z: {min_z:.1f}\t{max_z:.1f}')

meta['to_edge'] = np.minimum(
    np.minimum(meta['x']-min_x, max_x-meta['x']), 
    np.minimum(meta['y']-min_y, max_y-meta['y']), 
    np.minimum(meta['z']-min_z, max_z-meta['z']),
)
meta['cov'] = spot.sum(axis=1)

# bin data 
bins_8p = np.linspace(0,900,8+1).astype(int)
bins_4p = np.linspace(0,900,4+1).astype(int)
print(bins_8p, bins_4p)

meta['xb_8p'] = pd.cut(meta['x'], bins=bins_8p)
meta['yb_8p'] = pd.cut(meta['y'], bins=bins_8p)
meta['zb_8p'] = pd.cut(meta['z'], bins=bins_8p)

meta['xb_4p'] = pd.cut(meta['x'], bins=bins_4p)
meta['yb_4p'] = pd.cut(meta['y'], bins=bins_4p)
meta['zb_4p'] = pd.cut(meta['z'], bins=bins_4p)

In [ ]:
adata = anndata.AnnData(X=spot.values, obs=meta, var=var)
adata

In [ ]:
# remove outliers adata -> adata2
df = adata.obs
conds = [
    df['area'] > 500,
    df['area'] < 5000,
    df['x'] > min_x + 100,
    df['x'] < max_x - 100,
    
    df['y'] > min_y + 100,
    df['y'] < max_y - 100,
    
    df['z'] > min_z + 20,
    df['z'] < max_z - 20,
]
cond_all = np.ones(len(df)) > 0
for cond in conds:
    cond_all = np.logical_and(cond_all, cond)
    print(cond_all.sum())
print(f"Num cells before and after: {len(df)} -> {cond_all.sum()}")

# remove outliers
adata2 = adata[cond_all].copy()
adata2

In [ ]:
# separate the proj data adata2 -> adata3
adata3 = adata2[:,gene_idx].copy()
# adata3.obs[proj_idx] = np.array(adata2[:,proj_idx].X)
adata3.obs['cov_gene'] = np.array(adata3.X.sum(axis=1))

In [ ]:
# normalize the data and record
norm_data(adata2)
df_p2 = adata2.obs.copy()
df_p2[nrm_var_idx] = np.array(adata2.layers['nrm'])

norm_data(adata3)
df_p3 = adata3.obs.copy()
df_p3[nrm_gene_idx] = np.array(adata3.layers['nrm'])

# report 

In [ ]:
for idx in var_idx:
    val = adata[:,idx].X[:,0]
    print(f'{idx}\t{var_i2n.loc[idx]}\t{np.min(val):.1f}\t{np.max(val):.1f}\t{100*np.sum(val>0)/len(val):.2f}%')

In [ ]:
cols = ['x', 'y', 'z', 'area']
with sns.plotting_context('paper'):
    fig, axs = plt.subplots(4, 1, figsize=(1*6, 2*4))
    for ax, col in zip(axs, cols):
        sns.histplot(adata.obs[col], ax=ax)
        ax.set_xlabel(col)
    fig.subplots_adjust(hspace=0.5)
    plt.show()


In [ ]:
sns.scatterplot(data=adata.obs, x='area', y='cov', s=2, edgecolor='none')

In [ ]:
with sns.plotting_context('paper'):
    fig, axs = plt.subplots(2, 1, figsize=(10*1,4*2))
    ax = axs[0]
    sns.boxplot(data=adata.X, ax=ax)
    ax.set_xticklabels(adata.var.index.values, rotation=90) 
    ax.set_ylabel('counts')
    ax.set_xlabel('Genes')
    sns.despine(ax=ax)

    ax = axs[1]
    sns.boxplot(data=adata.X, ax=ax)
    ax.set_xticklabels(adata.var['name'].values, rotation=90)
    sns.despine(ax=ax)
    ax.set_ylim([0,50])
    ax.set_ylabel('counts')
    ax.set_xlabel('Genes')

In [ ]:
sns.scatterplot(data=adata3.obs, x='area', y='cov', s=2, edgecolor='none')
sns.scatterplot(data=adata3.obs, x='area', y='cov_gene', s=2, edgecolor='none')

In [ ]:
# sns.scatterplot(data=adata3.obs, x='area', y=proj_idx[0], s=2, edgecolor='none')

# z-sectioning visuals
- bin into zbin
- plot for each zbin

In [ ]:
import sys
sys.path.insert(0, '../')
import plotting

import importlib
importlib.reload(plotting)

from plotting import view_z_sections
from plotting import view_z_sections_4panels
from plotting import view_z_sections_labels
from plotting import gen_discrete_colors


In [ ]:
sp_x, sp_y = 'x', 'y'
col = gene_idx[0] # ftrs_order[0]
title = var_i2n[col]

view_z_sections_4panels(df_p2, sp_x, sp_y, col=col+'_nrm', cmap='Greys', title=title, flip_y=True)

In [ ]:
col = gene_idx[0] # ftrs_order[0]
title = var_i2n[col]

sp_x, sp_y, sp_z = 'x', 'y', 'zb_4p'
view_z_sections_4panels(df_p2, sp_x, sp_y, col=col+'_nrm', sp_z=sp_z, cmap='Greys', title=title, flip_y=True)

sp_x, sp_y, sp_z = 'x', 'z', 'yb_4p'
view_z_sections_4panels(df_p2, sp_x, sp_y, col=col+'_nrm', sp_z=sp_z, cmap='Greys', title=title, flip_y=True)

sp_x, sp_y, sp_z = 'z', 'y', 'xb_4p'
view_z_sections_4panels(df_p2, sp_x, sp_y, col=col+'_nrm', sp_z=sp_z, cmap='Greys', title=title, flip_y=True)

In [ ]:
sp_x, sp_y = 'x', 'z'
fig, axs = plt.subplots(5,4,figsize=(4*4,4*5), sharey=True, sharex=True)
cbar_ax = fig.add_axes([0.92, 0.5, 0.01, 0.2])
axs.flat[0].invert_yaxis()
for i, (col, ax) in enumerate(zip(var_idx[::-1], axs.flat)):
    x = df_p2[sp_x].values
    y = df_p2[sp_y].values
    c = df_p2[col+'_nrm'].values
    vmax = max(1e-5, np.percentile(c, 95))
    
    ax.axis('off')
    ax.grid(False)
    g = ax.scatter(x, y, c=c/vmax, s=5, edgecolor='none', cmap='Greys', vmax=1, vmin=-0.1) #vmax, vmin=-0.1*vmax)
    sns.despine(ax=ax)
    ax.set_title(var_i2n[col])
    ax.set_xlabel(sp_x)
    ax.set_ylabel(sp_y)
    ax.set_aspect('equal')

for ax in axs.flat[i+1:]:
    ax.axis('off')
    
fig.colorbar(g, cax=cbar_ax, label='Normed counts\n(0-95 perctl.)', aspect=5, shrink=0.3, ticks=[0, 1])
fig.subplots_adjust(hspace=0.1, wspace=0.02)

In [ ]:
sp_x, sp_y = 'y', 'z'
fig, axs = plt.subplots(5,4,figsize=(4*5,4*4), sharey=True, sharex=True)
cbar_ax = fig.add_axes([0.92, 0.5, 0.01, 0.2])
axs.flat[0].invert_yaxis()
for i, (col, ax) in enumerate(zip(var_idx[::-1], axs.flat)):
    x = df_p2[sp_x].values
    y = df_p2[sp_y].values
    c = df_p2[col+'_nrm'].values
    vmax = max(1e-5, np.percentile(c, 95))
    
    ax.axis('off')
    ax.grid(False)
    g = ax.scatter(x, y, c=c/vmax, s=5, edgecolor='none', cmap='Greys', vmax=1, vmin=-0.1) #vmax, vmin=-0.1*vmax)
    sns.despine(ax=ax)
    ax.set_title(var_i2n[col])
    ax.set_xlabel(sp_x)
    ax.set_ylabel(sp_y)
    ax.set_aspect('equal')

for ax in axs.flat[i+1:]:
    ax.axis('off')
    
fig.colorbar(g, cax=cbar_ax, label='Normed counts\n(0-95 perctl.)', aspect=5, shrink=0.3, ticks=[0, 1])
fig.subplots_adjust(hspace=0.1, wspace=0.02)

In [ ]:
sp_x, sp_y = 'x', 'y'
fig, axs = plt.subplots(5,4,figsize=(4*5,4*4), sharey=True, sharex=True)
cbar_ax = fig.add_axes([0.92, 0.5, 0.01, 0.2])
axs.flat[0].invert_yaxis()
for i, (col, ax) in enumerate(zip(var_idx[::-1], axs.flat)):
    x = df_p2[sp_x].values
    y = df_p2[sp_y].values
    c = df_p2[col+'_nrm'].values
    vmax = max(1e-5, np.percentile(c, 95))
    
    ax.axis('off')
    ax.grid(False)
    g = ax.scatter(x, y, c=c/vmax, s=5, edgecolor='none', cmap='Greys', vmax=1, vmin=-0.1) #vmax, vmin=-0.1*vmax)
    sns.despine(ax=ax)
    ax.set_title(var_i2n[col])
    ax.set_xlabel(sp_x)
    ax.set_ylabel(sp_y)
    ax.set_aspect('equal')

for ax in axs.flat[i+1:]:
    ax.axis('off')
    
fig.colorbar(g, cax=cbar_ax, label='Normed counts\n(0-95 perctl.)', aspect=5, shrink=0.3, ticks=[0, 1])
fig.subplots_adjust(hspace=0.1, wspace=0.02)

# hierarchical clustering

In [ ]:
mat_ftrs = pd.DataFrame(adata3.layers['zsc'], index=adata3.obs.index, columns=adata3.var.index)
mat_prjs = adata3.obs[proj_idx] 

# sch
col_link = sch.linkage(mat_ftrs.T, method='ward', metric='euclidean')
col_order = sch.dendrogram(col_link, no_plot=True)['leaves']

row_link = sch.linkage(mat_ftrs, method='ward', metric='euclidean')
row_order = sch.dendrogram(row_link, p=30, truncate_mode='none', no_plot=True)['leaves']

# get clusters
clsts1 = sch.fcluster(row_link, 200, criterion='distance')
clsts2 = sch.fcluster(row_link, 100, criterion='distance')
clsts3 = sch.fcluster(row_link,  50, criterion='distance')

print(clsts1.shape, np.unique(clsts1).shape)
print(clsts2.shape, np.unique(clsts2).shape)
print(clsts3.shape, np.unique(clsts3).shape)


# register
adata3.obs['clst_l1'] = clsts1
adata3.obs['clst_l2'] = clsts2
adata3.obs['clst_l3'] = clsts3

adata2.obs['clst_l1'] = clsts1
adata2.obs['clst_l2'] = clsts2
adata2.obs['clst_l3'] = clsts3


df_p3['clst_l1'] = clsts1
df_p3['clst_l2'] = clsts2
df_p3['clst_l3'] = clsts3

df_p2['clst_l1'] = clsts1
df_p2['clst_l2'] = clsts2
df_p2['clst_l3'] = clsts3

# colors
palette1, dismap1 = gen_discrete_colors(len(np.unique(clsts1)))
palette2, dismap2 = gen_discrete_colors(len(np.unique(clsts2)))
palette3, dismap3 = gen_discrete_colors(len(np.unique(clsts3)))

In [ ]:
fig = plt.figure(figsize=(10,15))
axd = fig.subplot_mosaic("A"*12+"BCDEF")
ax = axd["A"]
col_idx = mat_ftrs.columns[col_order].values.astype(str)
col_names = var_i2n[col_idx].values.astype(str)
xticklabels = np.char.add(np.char.add(col_names, ' '), col_idx)

sns.heatmap(
    mat_ftrs.iloc[row_order,col_order],
    yticklabels=False,
    xticklabels=xticklabels,
    cmap='coolwarm',
    cbar_kws=dict(shrink=0.5, location='bottom', label='zscore log2(1+norm expr.)'),
    vmax=3, vmin=-3,
    ax=ax,
)
ax.set_ylabel('Cells')
# ax.set_xticklabels()

for i, (ax, col) in enumerate(zip(
    (axd["B"], axd["C"], axd["D"]),
    ('clst_l1', 'clst_l2', 'clst_l3'),
    )):
    vec = adata3.obs[col].values
    sns.heatmap(
        adata3.obs[[col]].rename(columns={col: col.replace('clst_l', 'L')}).iloc[row_order],
        yticklabels=False,
        cmap='jet',
        cbar_kws=dict(location='bottom', aspect=5, label='', ticks=[]),
        ax=ax,
    )
    ax.set_ylabel('')
    positions = np.cumsum(np.unique(vec, return_counts=True)[1])
    ax.hlines(positions, xmin=-0.2, xmax=1, linewidth=1, linestyle='-', color='k', clip_on=False)
    for i, pos in enumerate(positions):
        ax.text(0, pos, i+1, color='white', fontsize=15)

# ax = axd["E"]
# col_idx = mat_prjs.columns.values.astype(str)
# col_names = var_i2n[col_idx].values.astype(str)
# xticklabels = np.char.add(np.char.add(col_names, ' '), col_idx)
# g = sns.heatmap(
#     mat_prjs.iloc[row_order],
#     yticklabels=False,
#     xticklabels=xticklabels,
#     cmap='Greys',
#     cbar_kws=dict(location='bottom', aspect=5, ),
#     ax=ax,
# )
# ax.tick_params(rotation=90)
# ax.set_ylabel('')
# colorbar = g.collections[0].colorbar
# colorbar.ax.tick_params(labelsize=10)  # Set fontsize to 12 points

ax = axd["F"]
g = sns.heatmap(
    adata3.obs[['z']].iloc[row_order], 
    yticklabels=False,
    cmap='Greys',
    cbar_kws=dict(location='bottom', aspect=5, label='um'),
    # vmin=0,
    ax=ax,
)
ax.set_ylabel('')
colorbar = g.collections[0].colorbar
colorbar.ax.tick_params(labelsize=10, rotation=90)  # Set fontsize to 12 points


In [ ]:
sp_x, sp_y = 'x', 'y'
fig, axs = plt.subplots(1,3,figsize=(4*3,5*1), sharey=True, sharex=True)
axs.flat[0].invert_yaxis()
# cbar_ax = fig.add_axes([0.92, 0.5, 0.01, 0.2])
for i, (col, palette, dismap, ax) in enumerate(zip(['clst_l1', 'clst_l2', 'clst_l3'], 
                                                   [ palette1,  palette2,  palette3],
                                                   [ dismap1,   dismap2,   dismap3],
                                                   axs.flat)):
    x = adata3.obs[sp_x].values
    y = adata3.obs[sp_y].values
    c = adata3.obs[col].values
    
    ax.grid(False)
    g = ax.scatter(x, y, c=pd.Series(palette)[c], s=5, edgecolor='none',)
    sns.despine(ax=ax)
    ax.set_title(col)
    ax.set_xlabel(sp_x)
    ax.set_ylabel(sp_y)
    ax.set_aspect('equal')
    ax.axis('off')
    fig.colorbar(dismap, ax=ax, orientation='horizontal', shrink=0.5, aspect=10)
    
fig.subplots_adjust(hspace=0.1, wspace=0.02)

In [ ]:
sp_x, sp_y = 'x', 'z'
fig, axs = plt.subplots(1,3,figsize=(4*3,5*1), sharey=True, sharex=True)
axs.flat[0].invert_yaxis()
# cbar_ax = fig.add_axes([0.92, 0.5, 0.01, 0.2])
for i, (col, palette, dismap, ax) in enumerate(zip(['clst_l1', 'clst_l2', 'clst_l3'], 
                                                   [ palette1,  palette2,  palette3],
                                                   [ dismap1,   dismap2,   dismap3],
                                                   axs.flat)):
    x = adata3.obs[sp_x].values
    y = adata3.obs[sp_y].values
    c = adata3.obs[col].values
    
    ax.grid(False)
    g = ax.scatter(x, y, c=pd.Series(palette)[c], s=5, edgecolor='none',)
    sns.despine(ax=ax)
    ax.set_title(col)
    ax.set_xlabel(sp_x)
    ax.set_ylabel(sp_y)
    ax.set_aspect('equal')
    ax.axis('off')
    fig.colorbar(dismap, ax=ax, orientation='horizontal', shrink=0.5, aspect=10)
    
fig.subplots_adjust(hspace=0.1, wspace=0.02)

In [ ]:
sp_x, sp_y = 'x', 'y'
col = 'clst_l1'
palette = palette1
dismap = dismap1
n = len(adata3.obs[col].unique())
nx = min(8,n)
ny = int((n+nx-1)/nx)
suptitle = 'L1'

fig, axs = plt.subplots(ny,nx,figsize=(3*nx,3*ny), sharey=True, sharex=True)
axs.flat[0].invert_yaxis()
for i, ax in enumerate(axs.flat):
    x = adata3.obs[sp_x].values
    y = adata3.obs[sp_y].values
    c = adata3.obs[col].values
    
    ax.grid(False)
    g = ax.scatter(x, y, c=[palette[_c] if i+1==_c else 'lightgray' for _c in c], s=5, edgecolor='none',)
    
    sns.despine(ax=ax)
    ax.set_title(f"C{i+1}")
    ax.set_xlabel(sp_x)
    ax.set_ylabel(sp_y)
    ax.set_aspect('equal')
    ax.axis('off')
    
fig.suptitle(suptitle)
fig.subplots_adjust(hspace=0.1, wspace=0.02)

In [ ]:
sp_x, sp_y = 'x', 'z'
col = 'clst_l1'
palette = palette1
dismap = dismap1
n = len(adata3.obs[col].unique())
nx = min(8,n)
ny = int((n+nx-1)/nx)
suptitle = 'L1'

fig, axs = plt.subplots(ny,nx,figsize=(3*nx,3*ny), sharey=True, sharex=True)
axs.flat[0].invert_yaxis()
for i, ax in enumerate(axs.flat):
    x = adata3.obs[sp_x].values
    y = adata3.obs[sp_y].values
    c = adata3.obs[col].values
    
    ax.grid(False)
    g = ax.scatter(x, y, c=[palette[_c] if i+1==_c else 'lightgray' for _c in c], s=5, edgecolor='none',)
    
    sns.despine(ax=ax)
    ax.set_title(f"C{i+1}")
    ax.set_xlabel(sp_x)
    ax.set_ylabel(sp_y)
    ax.set_aspect('equal')
    ax.axis('off')
    
fig.suptitle(suptitle)
fig.subplots_adjust(hspace=0.1, wspace=0.02)

In [ ]:
sp_x, sp_y = 'y', 'z'
col = 'clst_l1'
palette = palette1
dismap = dismap1
n = len(adata3.obs[col].unique())
nx = min(8,n)
ny = int((n+nx-1)/nx)
suptitle = 'L1'

fig, axs = plt.subplots(ny,nx,figsize=(3*nx,3*ny), sharey=True, sharex=True)
axs.flat[0].invert_yaxis()
for i, ax in enumerate(axs.flat):
    x = adata3.obs[sp_x].values
    y = adata3.obs[sp_y].values
    c = adata3.obs[col].values
    
    ax.grid(False)
    g = ax.scatter(x, y, c=[palette[_c] if i+1==_c else 'lightgray' for _c in c], s=5, edgecolor='none',)
    
    sns.despine(ax=ax)
    ax.set_title(f"C{i+1}")
    ax.set_xlabel(sp_x)
    ax.set_ylabel(sp_y)
    ax.set_aspect('equal')
    ax.axis('off')
    
fig.suptitle(suptitle)
fig.subplots_adjust(hspace=0.1, wspace=0.02)

In [ ]:
group = 'clst_l1'
dfmean = df_p2.groupby(group)[np.char.add(var_idx, '_nrm')].mean()
dfmean = basicu.zscore(dfmean, axis=0)

fig, ax = plt.subplots(figsize=(6,6))
sns.heatmap(dfmean.T, ax=ax, 
            yticklabels=var_i2n[var_idx], 
            cmap='coolwarm', cbar_kws=dict(label='zscore mean exp.', shrink=0.5, aspect=10))

In [ ]:
var_order_manual = pd.Series({
    'r2_c4': 'Adamts2',
    'r6_c1': 'Rfx3',
    'r7_c4': 'Grm8',
    
    'r6_c4': 'Baz1a',
    'r2_c0': 'Sorcs3',
    'r7_c2': 'Trpc6',
    'r2_c1': 'Kcnq5',
    
    'r6_c2': 'Epha10',
    'r7_c0': 'Kcnh5',
    'r6_c0': 'Ncam2',
    'r2_c2': 'Chrm2',
    
    'r7_c1': 'Slc17a7',
    'r1_c0': "RL (Cre)",
    'r1_c2': "LM (FlpO)",
    
})

In [ ]:
group = 'clst_l1'
dfmean = df_p2.groupby(group)[np.char.add(var_order_manual.index.values.astype(str), '_nrm')].mean()
dfmean = basicu.zscore(dfmean, axis=0)
# dfmean = dfmean.reindex(clst_order_manual)
yticklabels = var_order_manual.values


fig, ax = plt.subplots(figsize=(6,6))
sns.heatmap(dfmean.T, ax=ax, 
            yticklabels=yticklabels, 
            cmap='coolwarm', cbar_kws=dict(label='zscore mean exp.', shrink=0.5, aspect=10))

# remove outlier cells and redo

In [ ]:
remove_these = [1,2,5]
adata2v2 = adata2[~adata2.obs['clst_l1'].isin(remove_these)].copy()
adata3v2 = adata3[~adata3.obs['clst_l1'].isin(remove_these)].copy()

# remove_these_genes = ['Baz1a']
# adata2v2 = adata2v2[:,~adata2v2.var['name'].isin(remove_these_genes)]
# adata3v2 = adata3v2[:,~adata3v2.var['name'].isin(remove_these_genes)]

print(adata2v2.shape, adata3v2.shape)

In [ ]:
# normalize the data and record
norm_data(adata2v2)
norm_data(adata3v2)

In [ ]:
df_p2v2 = adata2v2.obs.copy()
df_p2v2[nrm_var_idx] = np.array(adata2v2.layers['nrm'])

df_p3v2 = adata3v2.obs.copy()
df_p3v2[nrm_gene_idx] = np.array(adata3v2.layers['nrm'])

In [ ]:
mat_ftrs = pd.DataFrame(adata3v2.layers['zsc'], index=adata3v2.obs.index, columns=adata3v2.var.index)
mat_prjs = adata3v2.obs[proj_idx] 

# sch
col_link = sch.linkage(mat_ftrs.T, method='ward', metric='euclidean')
col_order = sch.dendrogram(col_link, no_plot=True)['leaves']

row_link = sch.linkage(mat_ftrs, method='ward', metric='euclidean')
row_order = sch.dendrogram(row_link, p=30, truncate_mode='none', no_plot=True)['leaves']

# get clusters
clsts1 = sch.fcluster(row_link, 100, criterion='distance')
clsts2 = sch.fcluster(row_link,  70, criterion='distance')
clsts3 = sch.fcluster(row_link,  60, criterion='distance')

print(clsts1.shape, np.unique(clsts1).shape)
print(clsts2.shape, np.unique(clsts2).shape)
print(clsts3.shape, np.unique(clsts3).shape)


# register
adata3v2.obs['clst_l1'] = clsts1
adata3v2.obs['clst_l2'] = clsts2
adata3v2.obs['clst_l3'] = clsts3

adata2v2.obs['clst_l1'] = clsts1
adata2v2.obs['clst_l2'] = clsts2
adata2v2.obs['clst_l3'] = clsts3

df_p3v2['clst_l1'] = clsts1
df_p3v2['clst_l2'] = clsts2
df_p3v2['clst_l3'] = clsts3

df_p2v2['clst_l1'] = clsts1
df_p2v2['clst_l2'] = clsts2
df_p2v2['clst_l3'] = clsts3

# colors
palette1, dismap1 = gen_discrete_colors(len(np.unique(clsts1)))
palette2, dismap2 = gen_discrete_colors(len(np.unique(clsts2)))
palette3, dismap3 = gen_discrete_colors(len(np.unique(clsts3)))

In [ ]:
fig = plt.figure(figsize=(10,15))
axd = fig.subplot_mosaic("A"*12+"BCDEF")
ax = axd["A"]
col_idx = mat_ftrs.columns[col_order].values.astype(str)
col_names = var_i2n[col_idx].values.astype(str)
xticklabels = np.char.add(np.char.add(col_names, ' '), col_idx)

sns.heatmap(
    mat_ftrs.iloc[row_order,col_order],
    yticklabels=False,
    xticklabels=xticklabels,
    cmap='coolwarm',
    cbar_kws=dict(shrink=0.5, location='bottom', label='zscore log2(1+norm expr.)'),
    vmax=3, vmin=-3,
    ax=ax,
)
ax.set_ylabel('Cells')
# ax.set_xticklabels()

for i, (ax, col) in enumerate(zip(
    (axd["B"], axd["C"], axd["D"]),
    ('clst_l1', 'clst_l2', 'clst_l3'),
    )):
    vec = adata3v2.obs[col].values
    sns.heatmap(
        adata3v2.obs[[col]].rename(columns={col: col.replace('clst_l', 'L')}).iloc[row_order],
        yticklabels=False,
        cmap='jet',
        cbar_kws=dict(location='bottom', aspect=5, label='', ticks=[]),
        ax=ax,
    )
    ax.set_ylabel('')
    positions = np.cumsum(np.unique(vec, return_counts=True)[1])
    ax.hlines(positions, xmin=-0.2, xmax=1, linewidth=1, linestyle='-', color='k', clip_on=False)
    for i, pos in enumerate(positions):
        ax.text(0, pos, i+1, color='white', fontsize=15)

ax = axd["E"]
g = sns.heatmap(
    mat_prjs.iloc[row_order],
    yticklabels=False,
    cmap='Greys',
    cbar_kws=dict(location='bottom', aspect=5, ),
    ax=ax,
)
ax.set_ylabel('')
colorbar = g.collections[0].colorbar
colorbar.ax.tick_params(labelsize=10)  # Set fontsize to 12 points

ax = axd["F"]
g = sns.heatmap(
    adata3v2.obs[['y']].iloc[row_order], 
    yticklabels=False,
    cmap='Greys',
    cbar_kws=dict(location='bottom', aspect=5, label='um'),
    # vmin=0,
    ax=ax,
)
ax.set_ylabel('')
colorbar = g.collections[0].colorbar
colorbar.ax.tick_params(labelsize=10, rotation=90)  # Set fontsize to 12 points


In [ ]:
sp_x, sp_y = 'x', 'y'
fig, axs = plt.subplots(1,3,figsize=(4*3,5*1), sharey=True, sharex=True)
axs.flat[0].invert_yaxis()
# cbar_ax = fig.add_axes([0.92, 0.5, 0.01, 0.2])
for i, (col, palette, dismap, ax) in enumerate(zip(['clst_l1', 'clst_l2', 'clst_l3'], 
                                                   [ palette1,  palette2,  palette3],
                                                   [ dismap1,   dismap2,   dismap3],
                                                   axs.flat)):
    x = adata3v2.obs[sp_x].values
    y = adata3v2.obs[sp_y].values
    c = adata3v2.obs[col].values
    
    ax.grid(False)
    g = ax.scatter(x, y, c=pd.Series(palette)[c], s=5, edgecolor='none',)
    sns.despine(ax=ax)
    ax.set_title(col)
    ax.set_xlabel(sp_x)
    ax.set_ylabel(sp_y)
    ax.set_aspect('equal')
    ax.axis('off')
    fig.colorbar(dismap, ax=ax, orientation='horizontal', shrink=0.5, aspect=10)
    
fig.subplots_adjust(hspace=0.1, wspace=0.02)

In [ ]:
sp_x, sp_y = 'x', 'z'
fig, axs = plt.subplots(1,3,figsize=(4*3,5*1), sharey=True, sharex=True)
axs.flat[0].invert_yaxis()
# cbar_ax = fig.add_axes([0.92, 0.5, 0.01, 0.2])
for i, (col, palette, dismap, ax) in enumerate(zip(['clst_l1', 'clst_l2', 'clst_l3'], 
                                                   [ palette1,  palette2,  palette3],
                                                   [ dismap1,   dismap2,   dismap3],
                                                   axs.flat)):
    x = adata3v2.obs[sp_x].values
    y = adata3v2.obs[sp_y].values
    c = adata3v2.obs[col].values
    
    ax.grid(False)
    g = ax.scatter(x, y, c=pd.Series(palette)[c], s=5, edgecolor='none',)
    sns.despine(ax=ax)
    ax.set_title(col)
    ax.set_xlabel(sp_x)
    ax.set_ylabel(sp_y)
    ax.set_aspect('equal')
    ax.axis('off')
    fig.colorbar(dismap, ax=ax, orientation='horizontal', shrink=0.5, aspect=10)
    
fig.subplots_adjust(hspace=0.1, wspace=0.02)

In [ ]:
cluster_order = [3,4,2,1]
sp_x, sp_y = 'x', 'y'
col = 'clst_l1'
palette = palette1
dismap = dismap1
n = len(adata3v2.obs[col].unique())
nx = min(5,n)
ny = int((n+nx-1)/nx)
suptitle = 'L1'

fig, axs = plt.subplots(ny,nx,figsize=(3*nx,3*ny), sharey=True, sharex=True)
axs.flat[0].invert_yaxis()
for i, (clst, ax) in enumerate(zip(cluster_order, axs.flat)):
    x = adata3v2.obs[sp_x].values
    y = adata3v2.obs[sp_y].values
    c = adata3v2.obs[col].values
    
    ax.grid(False)
    g = ax.scatter(x, y, c=[palette[_c] if clst==_c else 'lightgray' for _c in c], s=5, edgecolor='none',)
    
    sns.despine(ax=ax)
    ax.set_title(f"C{clst}")
    ax.set_xlabel(sp_x)
    ax.set_ylabel(sp_y)
    ax.set_aspect('equal')
    ax.axis('off')
    
fig.suptitle(suptitle)
fig.subplots_adjust(hspace=0.1, wspace=0.02)

In [ ]:
cluster_order = [3,4,2,1]
sp_x, sp_y = 'x', 'z'
col = 'clst_l1'
palette = palette1
dismap = dismap1
n = len(adata3v2.obs[col].unique())
nx = min(5,n)
ny = int((n+nx-1)/nx)
suptitle = 'L1'

fig, axs = plt.subplots(ny,nx,figsize=(3*nx,3*ny), sharey=True, sharex=True)
axs.flat[0].invert_yaxis()
for i, (clst, ax) in enumerate(zip(cluster_order, axs.flat)):
    x = adata3v2.obs[sp_x].values
    y = adata3v2.obs[sp_y].values
    c = adata3v2.obs[col].values
    
    ax.grid(False)
    g = ax.scatter(x, y, c=[palette[_c] if clst==_c else 'lightgray' for _c in c], s=5, edgecolor='none',)
    
    sns.despine(ax=ax)
    ax.set_title(f"C{clst}")
    ax.set_xlabel(sp_x)
    ax.set_ylabel(sp_y)
    ax.set_aspect('equal')
    ax.axis('off')
    
fig.suptitle(suptitle)
fig.subplots_adjust(hspace=0.1, wspace=0.02)

In [ ]:
clst_order_manual = [3,4,2,1,]

In [ ]:
group = 'clst_l1'
dfmean = df_p2v2.groupby(group)[np.char.add(var_order_manual.index.values.astype(str), '_nrm')].mean()
dfmean = basicu.zscore(dfmean, axis=0).fillna(0)
dfmean = dfmean.reindex(clst_order_manual)
yticklabels = var_order_manual.values


fig, ax = plt.subplots(figsize=(6,6))
sns.heatmap(dfmean.T, ax=ax, 
            yticklabels=yticklabels, 
            cmap='coolwarm', cbar_kws=dict(label='zscore mean exp.', shrink=0.5, aspect=10))

In [ ]:
# clst_order_manual = [6,7,8,9,10,5,3,4,2,1] # 1+np.arange(n) # [3,4,2,1,]
clst_order_manual = [6,7,8,9,10,5,4,3,2,1] # 1+np.arange(n) # [3,4,2,1,]

In [ ]:
sp_x, sp_y = 'x', 'y'
col = 'clst_l2'
palette = palette2
dismap = dismap2
n = len(adata3v2.obs[col].unique())
nx = min(5,n)
ny = int((n+nx-1)/nx)
suptitle = 'L2'
cluster_order = clst_order_manual # 1+np.arange(n)

fig, axs = plt.subplots(ny,nx,figsize=(3*nx,3*ny), sharey=True, sharex=True)
axs.flat[0].invert_yaxis()
for i, (clst, ax) in enumerate(zip(cluster_order, axs.flat)):
    x = adata3v2.obs[sp_x].values
    y = adata3v2.obs[sp_y].values
    c = adata3v2.obs[col].values
    
    ax.grid(False)
    g = ax.scatter(x, y, c=[palette[_c] if clst==_c else 'lightgray' for _c in c], s=5, edgecolor='none',)
    
    sns.despine(ax=ax)
    ax.set_title(f"C{clst}")
    ax.set_xlabel(sp_x)
    ax.set_ylabel(sp_y)
    ax.set_aspect('equal')
    ax.axis('off')
    
fig.suptitle(suptitle)
fig.subplots_adjust(hspace=0.1, wspace=0.02)

In [ ]:
sp_x, sp_y = 'x', 'z'
col = 'clst_l2'
palette = palette2
dismap = dismap2
n = len(adata3v2.obs[col].unique())
nx = min(5,n)
ny = int((n+nx-1)/nx)
suptitle = 'L2'
# cluster_order = 1+np.arange(n)
# cluster_order = [6,7,8,9,10,4,5,3,2,1]
cluster_order = clst_order_manual # 1+np.arange(n)

fig, axs = plt.subplots(ny,nx,figsize=(3*nx,3*ny), sharey=True, sharex=True)
axs.flat[0].invert_yaxis()
for i, (clst, ax) in enumerate(zip(cluster_order, axs.flat)):
    x = adata3v2.obs[sp_x].values
    y = adata3v2.obs[sp_y].values
    c = adata3v2.obs[col].values
    
    ax.grid(False)
    g = ax.scatter(x, y, c=[palette[_c] if clst==_c else 'lightgray' for _c in c], s=5, edgecolor='none',)
    
    sns.despine(ax=ax)
    ax.set_title(f"C{clst}")
    ax.set_xlabel(sp_x)
    ax.set_ylabel(sp_y)
    ax.set_aspect('equal')
    ax.axis('off')
    
fig.suptitle(suptitle)
fig.subplots_adjust(hspace=0.1, wspace=0.02)

In [ ]:
var_order_manual_v2 = pd.Series({
    'r2_c4': 'Adamts2',
    # 'r6_c1': 'Rfx3',
    'r7_c4': 'Grm8',
    
    # 'r6_c4': 'Baz1a',
    'r2_c0': 'Sorcs3',
    'r7_c2': 'Trpc6',
    'r2_c1': 'Kcnq5',
    
    'r6_c2': 'Epha10',
    'r7_c0': 'Kcnh5',
    'r6_c0': 'Ncam2',
    'r2_c2': 'Chrm2',
    
    'r7_c1': 'Slc17a7',
    'r1_c0': "RL (Cre)",
    'r1_c2': "LM (FlpO)",
    
})
clst_order_manual_v2 = [6,8,9,10,5,4,3,2,1] # 1+np.arange(n) # [3,4,2,1,]

In [ ]:
group = 'clst_l2'
dfmean = df_p2v2.groupby(group)[np.char.add(var_order_manual_v2.index.values.astype(str), '_nrm')].mean()
dfmean = basicu.zscore(dfmean, axis=0).fillna(0)
dfmean = dfmean.reindex(clst_order_manual_v2)
yticklabels = var_order_manual_v2.values


fig, ax = plt.subplots(figsize=(6,6))
sns.heatmap(dfmean.T, ax=ax, 
            yticklabels=yticklabels, 
            cmap='coolwarm', cbar_kws=dict(label='zscore mean exp.', shrink=0.5, aspect=10))

# order cells by y-axis

In [ ]:
row_order = np.argsort(adata3v2.obs['y']).values # .sort_values().index
row_order

In [ ]:
fig = plt.figure(figsize=(10,15))
axd = fig.subplot_mosaic("A"*12+"BCDEF")
ax = axd["A"]
col_idx = mat_ftrs.columns[col_order].values.astype(str)
col_names = var_i2n[col_idx].values.astype(str)
xticklabels = np.char.add(np.char.add(col_names, ' '), col_idx)

sns.heatmap(
    mat_ftrs.iloc[row_order,col_order],
    yticklabels=False,
    xticklabels=xticklabels,
    cmap='coolwarm',
    cbar_kws=dict(shrink=0.5, location='bottom', label='zscore log2(1+norm expr.)'),
    vmax=3, vmin=-3,
    ax=ax,
)
ax.set_ylabel('Cells')
# ax.set_xticklabels()

for i, (ax, col) in enumerate(zip(
    (axd["B"], axd["C"], axd["D"]),
    ('clst_l1', 'clst_l2', 'clst_l3'),
    )):
    # vec = adata3v2.obs[col][row_order].values
    sns.heatmap(
        adata3v2.obs[[col]].rename(columns={col: col.replace('clst_l', 'L')}).iloc[row_order],
        yticklabels=False,
        cmap='jet',
        cbar_kws=dict(location='bottom', aspect=5, label='', ticks=[]),
        ax=ax,
    )
    ax.set_ylabel('')
    # positions = np.cumsum(np.unique(vec, return_counts=True)[1])
    # ax.hlines(positions, xmin=-0.2, xmax=1, linewidth=1, linestyle='-', color='k', clip_on=False)
    # for i, pos in enumerate(positions):
    #     ax.text(0, pos, i+1, color='white', fontsize=15)

ax = axd["E"]
g = sns.heatmap(
    mat_prjs.iloc[row_order],
    yticklabels=False,
    cmap='Greys',
    cbar_kws=dict(location='bottom', aspect=5, ),
    ax=ax,
)
ax.set_ylabel('')
colorbar = g.collections[0].colorbar
colorbar.ax.tick_params(labelsize=10)  # Set fontsize to 12 points

ax = axd["F"]
g = sns.heatmap(
    adata3v2.obs[['y']].iloc[row_order], 
    yticklabels=False,
    cmap='Greys',
    cbar_kws=dict(location='bottom', aspect=5, label='um'),
    # vmin=0,
    ax=ax,
)
ax.set_ylabel('')
colorbar = g.collections[0].colorbar
colorbar.ax.tick_params(labelsize=10, rotation=90)  # Set fontsize to 12 points

# co-expression, clustering, cluster enrichment of expression

In [ ]:
sp_x, sp_y = 'x', 'y'
col = 'clst_l1'
view_z_sections_labels(df_p3v2, sp_x, sp_y, col, palette=palette1, title=None, flip_y=True)

In [ ]:
sp_x, sp_y = 'x', 'y'
col = 'clst_l2'
view_z_sections_labels(df_p3v2, sp_x, sp_y, col, palette=palette2, title=None, flip_y=True)

# pseudotime ordering

In [ ]:
import scanpy as sc
from scipy.stats import spearmanr

In [ ]:
pca = PCA(n_components=10)
pcs = pca.fit_transform(mat_ftrs.values)
pcs.shape

In [ ]:
plt.plot(pca.explained_variance_ratio_, '-o')

In [ ]:
df_p2v2['pc1'] = pcs[:,0]
df_p2v2['pc2'] = pcs[:,1]
df_p2v2['pc3'] = pcs[:,2]
df_p2v2['pc4'] = pcs[:,3]

In [ ]:
adata3v2.obsm['X_pca'] = pcs 
sc.pp.neighbors(adata3v2, n_neighbors=30, use_rep='X_pca')
sc.tl.diffmap(adata3v2)
adata3v2.uns['iroot'] = np.argmin(pcs[:,0]-pcs[:,1])
sc.tl.dpt(adata3v2)

df_p2v2['ptime'] = adata3v2.obs['dpt_pseudotime'].values

In [ ]:
fig, ax = plt.subplots()
sns.scatterplot(data=df_p2v2, x='pc1', y='pc2', hue='clst_l2', palette=palette2, s=5, edgecolor='none', ax=ax)
ax.legend(bbox_to_anchor=(1,1))
ax.grid(False)
plt.show()

fig, ax = plt.subplots()
sns.scatterplot(data=df_p2v2, x='pc3', y='pc4', hue='clst_l2', palette=palette2, s=5, edgecolor='none', ax=ax)
ax.legend(bbox_to_anchor=(1,1))
ax.grid(False)
plt.show()

In [ ]:
fig, ax = plt.subplots()
sns.scatterplot(data=df_p2v2, x='pc1', y='pc2', hue='ptime', s=5, edgecolor='none', ax=ax)
ax.legend(bbox_to_anchor=(1,1))
ax.grid(False)
plt.show()

In [ ]:
fig, ax = plt.subplots()
sns.scatterplot(data=df_p2v2, x='pc1', y='pc2', hue='y', s=5, edgecolor='none', ax=ax)
ax.legend(bbox_to_anchor=(1,1))
ax.grid(False)
plt.show()

In [ ]:
x = 'y'
y = 'ptime'

fig, ax = plt.subplots()
a = df_p2v2[x] 
b = df_p2v2[y] 
r, p = spearmanr(a, b)
print(r, p)
sns.scatterplot(data=df_p2v2, x=x, y=y, hue='clst_l2', palette=palette2, s=5, edgecolor='none', ax=ax)
ax.set_title(f'Spearman r={r:.2f}, p={p:.1e}')
ax.legend(bbox_to_anchor=(1,1))
ax.grid(False)
plt.show()

In [ ]:
x = 'y'
y = 'ptime'
fig, axs = plt.subplots(4,4,figsize=(4*4,5*4), sharex=True, sharey=True)
for col, ax in zip(var_idx, axs.flat):
    hue = col+'_nrm'
    c = df_p2v2[hue].values
    vmax = np.percentile(c,95) 
    vmin = -0.1*vmax # np.percentile(c,0) 
    g = sns.scatterplot(data=df_p2v2, x=x, y=y, c=c, cmap='gray_r', # hue=hue, 
                    vmin=vmin, vmax=vmax, s=5, edgecolor='none', ax=ax)
    ax.set_title(col+'_'+var_i2n[col])
    ax.grid(False)
    sns.despine(ax=ax)
plt.show()

In [ ]:
row_order = np.argsort(df_p2v2['ptime'].values)

In [ ]:
col_order = basicu.get_index_from_array(mat_ftrs.columns, var_order_manual.index.values[:-1])

In [ ]:

fig = plt.figure(figsize=(20,8))
axd = fig.subplot_mosaic("A\n"*12+"B\nC\nD\nE\nF")
ax = axd["A"]
col_idx = mat_ftrs.columns[col_order].values.astype(str)
col_names = var_i2n[col_idx].values.astype(str)
yticklabels = np.char.add(np.char.add(col_names, ' '), col_idx)

sns.heatmap(
    mat_ftrs.iloc[row_order,col_order].T,
    xticklabels=False,
    yticklabels=yticklabels,
    cmap='coolwarm',
    cbar_kws=dict(shrink=0.5, label='zscore log2(1+norm expr.)'),
    vmax=3, vmin=-3,
    ax=ax,
)
ax.set_ylabel('Cells')
# ax.set_xticklabels()

for i, (ax, col) in enumerate(zip(
    (axd["B"], axd["C"], axd["D"]),
    ('clst_l1', 'clst_l2', 'clst_l3'),
    )):
    # vec = adata3v2.obs[col][row_order].values
    sns.heatmap(
        adata3v2.obs[[col]].rename(columns={col: col.replace('clst_l', 'L')}).iloc[row_order].T,
        xticklabels=False,
        cmap='jet',
        cbar_kws=dict(aspect=5, label='', ticks=[]),
        ax=ax,
    )
    ax.set_ylabel('')
    # positions = np.cumsum(np.unique(vec, return_counts=True)[1])
    # ax.hlines(positions, xmin=-0.2, xmax=1, linewidth=1, linestyle='-', color='k', clip_on=False)
    # for i, pos in enumerate(positions):
    #     ax.text(0, pos, i+1, color='white', fontsize=15)

ax = axd["E"]
g = sns.heatmap(
    mat_prjs.iloc[row_order].T,
    xticklabels=False,
    cmap='Greys',
    cbar_kws=dict(aspect=5, ),
    ax=ax,
)
ax.set_ylabel('')
colorbar = g.collections[0].colorbar
colorbar.ax.tick_params(labelsize=10)  # Set fontsize to 12 points

ax = axd["F"]
g = sns.heatmap(
    adata3v2.obs[['y']].iloc[row_order].T, 
    xticklabels=False,
    cmap='Greys',
    cbar_kws=dict(aspect=5, label='um'),
    # vmin=0,
    ax=ax,
)
ax.set_ylabel('')
colorbar = g.collections[0].colorbar
colorbar.ax.tick_params(labelsize=10)  # Set fontsize to 12 points